In [19]:
import numpy as np
from skimage.io import imshow
import matplotlib.pyplot as plt

from skimage.transform import resize
from skimage.io import imread
import os

def color_map(N=256, normalized=True):
    def bitget(byteval, idx):
        return ((byteval & (1 << idx)) != 0)

    dtype = 'float32' if normalized else 'uint8'
    cmap = np.zeros((N, 3), dtype=dtype)
    for i in range(N):
        r = g = b = 0
        c = i
        for j in range(8):
            r = r | (bitget(c, 0) << 7-j)
            g = g | (bitget(c, 1) << 7-j)
            b = b | (bitget(c, 2) << 7-j)
            c = c >> 3

        cmap[i] = np.array([r, g, b])

    cmap = cmap/255 if normalized else cmap
    return cmap


def color_map_viz():
    labels = ['background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor', 'void']
    nclasses = 21
    row_size = 50
    col_size = 500
    cmap = color_map()
    array = np.empty((row_size*(nclasses+1), col_size, cmap.shape[1]), dtype=cmap.dtype)
    for i in range(nclasses):
        array[i*row_size:i*row_size+row_size, :] = cmap[i]
    array[nclasses*row_size:nclasses*row_size+row_size, :] = cmap[-1]

    print([row_size*i+row_size/2 for i in range(nclasses+1)])
    print(array[0:21])
    
    imshow(array)
    plt.yticks([row_size*i+row_size/2 for i in range(nclasses+1)], labels)
    plt.xticks([])
    plt.show()
    return array


def color_map_label():
    labels = ['background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor', 'void']
    nclasses = 21
    cmap = color_map()
    label_colors = np.empty((nclasses + 1, 3), dtype=cmap.dtype)  # +1 for the 'void' label

    for i in range(nclasses + 1):
        label_colors[i] = cmap[i] if i < nclasses else cmap[-1]

    return label_colors

The history saving thread hit an unexpected error (OperationalError('no such table: history')).History will not be written to the database.


In [20]:
import numpy as np
from skimage.transform import resize
from skimage.io import imread
import os
from concurrent.futures import ThreadPoolExecutor

def preprocess_and_save(image_path, mask_path, cmap, output_size=(224, 224), output_dir='output'):
    # Load and resize image
    image = imread(image_path)
    image = resize(image, output_size, anti_aliasing=True) / 255.0  # Normalize to [0, 1]

    # Load and resize mask
    mask_image = imread(mask_path)
    mask_image = resize(mask_image, output_size, anti_aliasing=False, preserve_range=True).astype(int)

    # Initialize 22-channel binary mask
    channels = np.zeros((*output_size, 22), dtype=np.float32)

    # Vectorized mask processing
    for idx, val in enumerate(cmap):
        try:
            channels[:, :, idx] = np.all(mask_image == val, axis=-1)
        except Exception as e:
            print(e)
        except:
            print('amme kharab')

    # Save processed image and mask using numpy.save
    image_filename = os.path.join(output_dir, os.path.basename(image_path) + '_image.npy')
    mask_filename = os.path.join(output_dir, os.path.basename(mask_path) + '_mask.npy')
    np.save(image_filename, image)
    np.save(mask_filename, channels)
    return image_filename

def process_dataset(image_dir, mask_dir, cmap):
    file_names = os.listdir(image_dir)
    if not os.path.exists('output'):
        os.makedirs('output')

    # Parallel processing
    with ThreadPoolExecutor(max_workers=16) as executor:
        for file_name in file_names:
            if file_name.endswith('.jpg'):
                mask_name = file_name[:-4] + '.png'  # Change extension for mask
                mask_path = os.path.join(mask_dir, mask_name)
                image_path = os.path.join(image_dir, file_name)

                if os.path.exists(mask_path):
                    executor.submit(preprocess_and_save, image_path, mask_path, cmap)

# Assume color_map function is defined and provides the cmap array
cmap = color_map()
cmap = color_map_label()

image_dir = 'data/VOC2007/JPEGImages'
mask_dir = 'data/VOC2007/SegmentationClass'
process_dataset(image_dir, mask_dir, cmap)


22
